# Shot analysis for calibration data
This is the analysis of the resistor/empty shots to get the values needed for calibrating all other shots. Different values are needed for the two different shunt resistors used, as they required different ADC sensitivities and therefore had slightly different offsets. Note that the internal resistances, cable resistances and feedthrough resistances are separate from the ADC offsets. 

In [1]:
%matplotlib tk
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import sys
import glob
import os
import pathlib as pth
sys.path.append('/home/jleland/Coding/Projects/flopter')
# import analysis.scripts.magnum.magnum_postrun_initial_analysis as mpia
import flopter.magnum.magoptoffline as mg
import flopter.core.fitters as fts

In [40]:
folders = ['2019-05-28_Leland/', '2019-06-05_Leland/']
mg.Magoptoffline._FOLDER_STRUCTURE = 'data/external/magnum/'
files = []
file_folders = []
for folder1 in folders:

    os.chdir(mg.Magoptoffline.get_data_path() / folder1)
    files.extend(glob.glob('*.adc'))
    file_folders.extend([folder1] * len(glob.glob('*.adc')))
files.sort()


In [41]:
for i, f in enumerate(files):
    print(i, f)

0 2019-05-28 10h 30m 08s TT_06695992307612752513.adc
1 2019-05-28 12h 14m 30s TT_06696019202114007511.adc
2 2019-05-28 12h 21m 44s TT_06696021067838019493.adc
3 2019-05-28 12h 48m 38s TT_06696027997221515869.adc
4 2019-05-28 14h 20m 45s TT_06696051736256851667.adc
5 2019-05-28 15h 49m 45s TT_06696074692831398075.adc
6 2019-05-28 15h 56m 35s TT_06696076438102672256.adc
7 2019-05-28 15h 58m 31s TT_06696076936923400130.adc
8 2019-05-28 16h 19m 55s TT_06696082451310700227.adc
9 2019-05-28 16h 22m 11s TT_06696082985479260440.adc
10 2019-05-28 16h 25m 58s TT_06696084010909391205.adc
11 2019-05-28 16h 27m 39s TT_06696084446035357724.adc
12 2019-05-28 16h 31m 10s TT_06696085352966572043.adc
13 2019-05-28 16h 32m 42s TT_06696085746216338023.adc
14 2019-05-28 17h 14m 14s TT_06696096449516280973.adc
15 2019-05-28 17h 16m 07s TT_06696096934807785572.adc
16 2019-05-28 17h 18m 00s TT_06696097420614687980.adc
17 2019-05-28 17h 20m 21s TT_06696098027540441792.adc
18 2019-05-28 17h 22m 35s TT_066960986

In [63]:
feedthrough_resistance = 1.25
internal_resistance = 6.09
CABLE_RESISTANCES = [1.7, 1.7, 1.7, 1.6]
PROBE_RESISTANCES = {
    'S': 1.0,
    'L': 1.0,
    'B': 1.0,
    'R': 1.8
}
baseline_resistance = feedthrough_resistance + internal_resistance + 1.0 + 1.7

### ADC Files Infos for resistor

In [64]:
adc_file = files[2]
folder = folders[0]
print('"{}" \t\t "{}"'.format(folder, adc_file))

"2019-05-28_Leland/" 		 "2019-05-28 12h 21m 44s TT_06696021067838019493.adc"


## Plots and Measurements of the 103.2 ohm resistor

In [67]:
# uses S and L probes, 10ohm shunt resistor
dsr = 10

# The resistance through the circuit is only the cable + internal resistance as no probes were used (and therefore no feedthrough either)
cabling_resistance = [1.7 + 6.09, 1.7 + 6.09]
# cabling_resistance = 1.7 + 6.09

# Create magopter object
magopter = mg.Magoptoffline(folder, adc_file, shunt_resistor=10.0, cabling_resistance=cabling_resistance)
magopter._VOLTAGE_CHANNEL = 3
magopter._PROBE_CHANNEL_3 = 4
magopter._PROBE_CHANNEL_4 = 5
magopter.prepare(down_sampling_rate=dsr, roi_b_plasma=True, filter_arcs_fl=False, crit_freq=None, crit_ampl=None)

print('0: {}, 1: {}'.format(len(magopter.iv_arrs[0]), len(magopter.iv_arrs[1])))

Running an offline magopter object, operating in offline mode.
0: 1997, 1: 1997


In [68]:
fig, ax = plt.subplots(2, sharex=True)

ax[0].plot(magopter.raw_time, magopter.voltage[0])
ax[0].plot(magopter.raw_time, magopter.voltage[1])
ax[0].set_ylabel(r'$V_P$ (V)')
ax[1].plot(magopter.raw_time, magopter.current[0])
ax[1].plot(magopter.raw_time, magopter.current[1])
ax[1].set_ylabel(r'$I$ (A)')
plt.show()

In [69]:
plt.figure()
plt.plot(magopter.voltage[0][:200], magopter.current[0][:200])

In [47]:
print(len(magopter.voltage[0]))
plt.figure()
plt.plot(magopter.voltage[0][:100000], magopter.current[0][:100000])
plt.show()

100000


In [70]:
ds_full = magopter.to_xarray(['resistor', 'empty']) 

In [71]:
fig, ax = plt.subplots(2, sharex=True)
ds_full.mean(['sweep']).set_coords('voltage').sel(probe='resistor')['current'].plot.line(x='voltage', hue='direction', ax=ax[0])
ds_full.mean(['sweep']).set_coords('voltage').sel(probe='empty')['current'].plot.line(x='voltage', hue='direction', ax=ax[1])
plt.show()

In [72]:
line_defaults = {
    'color': 'grey',
    'linewidth': .9,
    'linestyle': '--'
}

 Re-ran the following sections to see what the offset current was for the 10ohm shunt resistor. Found a value of ~0.001 (but in opposite directions for either coax) which makes negligible difference to IV characteristics which are, in general, ~2 orders of magnitude greater than this and therefore this would be lost in the noise. 

In [83]:
sl_fitter = fts.StraightLineFitter()
# fit_data = sl_fitter.fit(0.946 * magopter.voltage[0], magopter.current[0] - 0)
fit_data = sl_fitter.fit(magopter.voltage[0], magopter.current[0] - -0.000925)
resistance = 1 / fit_data.get_fit_params()[0].value
d_resist = (resistance * fit_data.get_fit_params()[0].error) / fit_data.get_fit_params()[0].value
print(f'Resistance: {resistance:.5g} +- {d_resist:.2g}')
print(f'Offset: {fit_data.get_fit_params()[1].value:.3g} +- {fit_data.get_fit_params()[1].error:.1g}')


Resistance: 105.2 +- 0.01
Offset: -3.28e-07 +- 5e-05


In [85]:
empty_fit_data = sl_fitter.fit(magopter.voltage[1], magopter.current[1] - 0.0011659376454164726)
empty_r = 1 / empty_fit_data.get_param('m')
d_empty_r = (empty_r * empty_fit_data.get_param('m', errors_fl=True).error) / empty_fit_data.get_param('m')

print(f'Resistance: {empty_r:.5g} +- {d_empty_r:.2g}')
print(f'Offset: {empty_fit_data.get_param("y_0"):.3g} +- {empty_fit_data.get_param("y_0", errors_fl=True).error:.1g}')
print(f'mean: {np.mean(magopter.current[1] - 0.0011659376454164726)}')

Resistance: 3.3203e+06 +- 7.9e+05
Offset: 1.29e-05 +- 4e-06
mean: 2.148281552649678e-19


In [77]:
res = []
off = []
offsets = np.linspace(-10, 10, 21)

for offset in offsets:
    fit_data = sl_fitter.fit(magopter.voltage[0], magopter.current[0] - offset)
    res.append(1 / fit_data.get_fit_params()[0].value)
    off.append(fit_data.get_fit_params()[1].value)

fig, ax = plt.subplots(2)
ax[0].plot(offsets, res)
ax[1].plot(offsets, off)

In [82]:
res = []
d_res = []
off = []
scalars = np.linspace(0.8, 1.2, 21)

for scalar in scalars:
    fit_data = sl_fitter.fit(magopter.voltage[0], magopter.current[0] * scalar)
    resist = 1 / fit_data.get_fit_params()[0].value
    d_resist = (resistance * fit_data.get_fit_params()[0].error) / fit_data.get_fit_params()[0].value
    res.append(resist)
    d_res.append(d_resist)
    off.append(fit_data.get_fit_params()[1].value)

fig, ax = plt.subplots(2)
ax[0].errorbar(scalars, res, yerr=d_res)
ax[0].axhline(105.2, **line_defaults)
ax[1].plot(scalars, off)

In [61]:
for i in range(2):
    print(f'{i}) {magopter.current[i].mean()} : {magopter.current[i].std()}')

0) -0.37783118198469995 : 0.26971954757501665
1) 0.0011659376454164726 : 0.0006954487134194552


In [80]:
res = []
d_res = []
off = []
scalars = np.linspace(0.8, 1.2, 21)

for scalar in scalars:
    fit_data = sl_fitter.fit(magopter.voltage[0] * scalar, magopter.current[0])
    resist = 1 / fit_data.get_fit_params()[0].value
    d_resist = (resistance * fit_data.get_fit_params()[0].error) / fit_data.get_fit_params()[0].value
    res.append(resist)
    d_res.append(d_resist)
    off.append(fit_data.get_fit_params()[1].value)

fig, ax = plt.subplots(2)
ax[0].errorbar(scalars, res, yerr=d_res)
ax[0].axhline(105.2, **line_defaults)
ax[1].plot(scalars, off)

# Analysis of Empty Shot
This is an analysis of an empty shot (no plasma but through the probes) which should help show ADC offsets and internal resistances

In [19]:
# 0		-	2019-05-28 09:29:58	6695992307612752513
# 272	-	2019-06-05 09:58:51	6698968431022989763
# 334	-	2019-06-05 15:24:00	6699052221659850975
timestamps_oi = ['6695992307612752513', '6698968431022989763', '6699052221659850975']


In [20]:
indices_oi = []
for i, f in enumerate(files):
    if any(timestamp in f for timestamp in timestamps_oi):
        print(i, f)
        indices_oi.append(i)
print(indices_oi)

0 2019-05-28 10h 30m 08s TT_06695992307612752513.adc
33 2019-06-05 10h 59m 01s TT_06698968431022989763.adc
95 2019-06-05 16h 24m 22s TT_06699052221659850975.adc
[0, 33, 95]


In [21]:
adc_file_0 = files[indices_oi[0]]
adc_file_1 = files[indices_oi[1]]
adc_file_2 = files[indices_oi[2]]
print(f'{folders[0]}{adc_file_0}')
print(f'{folders[1]}{adc_file_1}')
print(f'{folders[1]}{adc_file_2}')

2019-05-28_Leland/2019-05-28 10h 30m 08s TT_06695992307612752513.adc
2019-06-05_Leland/2019-06-05 10h 59m 01s TT_06698968431022989763.adc
2019-06-05_Leland/2019-06-05 16h 24m 22s TT_06699052221659850975.adc


In [23]:
folder = folders[0]
adc_file = adc_file_0
shunt_resistor = 10.0

dsr = 10
# cabling_resistance = [1.2 + 0.7, 1.2 + 0.7]
cabling_resistance = [baseline_resistance, baseline_resistance]

print(folder, adc_file, shunt_resistor, cabling_resistance)


# Create magopter object
magopter = mg.Magoptoffline(folder, adc_file, shunt_resistor=1.0, cabling_resistance=cabling_resistance)
magopter._VOLTAGE_CHANNEL = 3
magopter._PROBE_CHANNEL_3 = 4
magopter._PROBE_CHANNEL_4 = 5
magopter.prepare(down_sampling_rate=dsr, roi_b_plasma=True, filter_arcs_fl=False, crit_freq=None, crit_ampl=None)

print('0: {}, 1: {}'.format(len(magopter.iv_arrs[0]), len(magopter.iv_arrs[1])))

2019-05-28_Leland/ 2019-05-28 10h 30m 08s TT_06695992307612752513.adc 10.0 [1.9, 1.9]
Running an offline magopter object, operating in offline mode.
0: 1992, 1: 1992


In [24]:
fig, ax = plt.subplots(2, sharex=True)

ax[0].plot(magopter.raw_time, magopter.voltage[0])
ax[0].plot(magopter.raw_time, magopter.voltage[1])
ax[0].set_ylabel(r'$V_P$ (V)')
ax[1].plot(magopter.raw_time, magopter.current[0])
ax[1].plot(magopter.raw_time, magopter.current[1])
ax[1].set_ylabel(r'$I$ (A)')
plt.show()

In [25]:
print(np.mean(magopter.current[0]))
print(np.mean(magopter.current[1]))

-0.04813597672664584
-0.03822868483989308


In [26]:
ds_full = magopter.to_xarray(['resistor', 'empty'])
fig, ax = plt.subplots(2, sharex=True)
ds_full.mean(['sweep']).set_coords('voltage').sel(probe='resistor')['current'].plot.line(x='voltage', hue='direction', ax=ax[0])
ds_full.mean(['sweep']).set_coords('voltage').sel(probe='empty')['current'].plot.line(x='voltage', hue='direction', ax=ax[1])
plt.show()

In [28]:
folders_oi = ['2019-05-28_Leland/', '2019-06-05_Leland/', '2019-06-05_Leland/']
res_oi = [10.0, 1., 1.]
p_d = [
    ['empty1', 'empty2'],
    ['S', 'L'],
    ['R', 'B'],
]

for i, index in enumerate(indices_oi):
    adc_file = files[index]
    folder = folders_oi[i]
    shunt_resistor = res_oi[i]
    
    dsr = 10
    cabling_resistance = [1.2 + 0.7, 1.2 + 0.7]
    
    magopter = mg.Magoptoffline(folder, adc_file, shunt_resistor=1.0, cabling_resistance=cabling_resistance)
    magopter._VOLTAGE_CHANNEL = 3
    magopter._PROBE_CHANNEL_3 = 4
    magopter._PROBE_CHANNEL_4 = 5
    magopter.prepare(down_sampling_rate=dsr, roi_b_plasma=True, filter_arcs_fl=False, crit_freq=None, crit_ampl=None)
    
    probes = p_d[i]
    print(f'{i}, {probes} \n'
          f'\tchannel_0 mean: {np.mean(magopter.current[0])}, \n'
          f'\tchannel_1 mean: {np.mean(magopter.current[1])}')
    
    ds_full = magopter.to_xarray(probes)
    
    fig, ax = plt.subplots(2, sharex=True)
    ds_full.mean(['sweep']).set_coords('voltage').sel(probe=probes[0])['current'].plot.line(x='voltage', hue='direction', ax=ax[0])
    ds_full.mean(['sweep']).set_coords('voltage').sel(probe=probes[1])['current'].plot.line(x='voltage', hue='direction', ax=ax[1])
    
plt.show()

Running an offline magopter object, operating in offline mode.
0, ['empty1', 'empty2'] 
	channel_0 mean: -0.04813597672664584, 
	channel_1 mean: -0.03822868483989308
Running an offline magopter object, operating in offline mode.
1, ['S', 'L'] 
	channel_0 mean: 2.8810287489022813e-18, 
	channel_1 mean: -2.425171174991192e-18
Running an offline magopter object, operating in offline mode.
2, ['R', 'B'] 
	channel_0 mean: 0.0005036455711081326, 
	channel_1 mean: 0.0010024194546457987


In [29]:
import scipy.signal as sig

In [30]:
sig.savgol_filter(magopter.current[0], 21, 5)

array([ 0.00249896,  0.00258628,  0.00214627, ...,  0.00137535,
        0.00070056, -0.00047627])